#Tutorial on CNN
We try to build a DNN that recognize if an image contains a rectangle or a circle, count how many rectangle and circles are there or find where in the image the rectangle/circle is.

## Import useful stuff

In [ ]:
from keras.layers import #FILL ME# Which layers will be needed in a CNN ?
from keras.models import Model
import numpy as np
from math import *
from matplotlib import pyplot as plt

## Lets generate some data

We now generate ourself some images with a circle or a rectangle, of random color, in a random position.

Three different modes of generating the images are implemented:

*   A single shape per figure
*   Multiple shapes mixed in each figure
*   A single figure keeping track of the "bounding box"



### Additional Exercise
1. Try adding some random noise in the image background
2. Try adding more classes such e.g. Lines or Ellipses


In [ ]:
import cv2

simple =False #single shape per figure
mixed  =False #multi shapes per figure
withBB =True  #one shape with bounding boxes

def background():
  return np.zeros((64,64,3), np.uint8) #EX# How would you change this to add some background noise?

def randomColor():
  return (int(np.random.rand()*128+128),int(np.random.rand()*128+128),int(np.random.rand()*128+128))

def drawCircle(c,x,y,r):
  img = background()
  cv2.circle(img,(x,y),r,c, -1)
  return img,x-r,y-r,x+r,y+r   #return image and bounding box

def genCircle():
  return drawCircle(randomColor(),int(np.random.rand()*50)+10,int(np.random.rand()*50)+10,
                    int(np.random.rand()*6)+3)

def drawRectangle(c,x,y,w,h):
  img = background()
  cv2.rectangle(img,(x,y),((x+w),(y+h)), c, -1)
  return img,x,y,x+w,y+h #return image and bounding box

def genRectangle():
  return drawRectangle(randomColor(),int(np.random.rand()*40)+10,int(np.random.rand()*40)+10,
                       int(np.random.rand()*12)+5,int(np.random.rand()*12)+5)

def genN(f,i): # generate multiple shapes
  img = np.zeros((64,64,3), np.uint8)
  for x in range(i):
    img+=f()[0] #discard bb info, take only image
  return img

nsamples=10000

#produce figures with either a rectangle or a circle
if simple :
  targets=np.random.rand(nsamples)>0.5 #np.array([np.random.rand()>0.5 for x in range(nsamples) ])
  images=np.array([genCircle()[0] if targets[x] else genRectangle()[0] for x in range(nsamples)])

#produce figure with n rectangles and m circles
if mixed:
  targets=np.array([(int(np.random.rand()*4),int(np.random.rand()*4)) for x in range(nsamples) ])
  images=np.array([genN(genRectangle,targets[x,0])+genN(genCircle,targets[x,1]) for x in range(nsamples)])

if withBB :
#produce figures with either a rectangle or a circle
  targets=np.random.rand(nsamples)>0.5 #np.array([np.random.rand()>0.5 for x in range(nsamples) ])
  imagesWithBB=[genCircle() if targets[x] else genRectangle() for x in range(nsamples)]
  images=np.array([imagesWithBB[x][0] for x in range(nsamples)])
  boundingBoxes=np.array([imagesWithBB[x][1:] for x in range(nsamples)])


Let's show a few of the generated images

In [ ]:

#FILL ME# print the shape of the images and target data
#FILL ME# print the the first N entries of the target data
print(targets[:4])
#show the first 4 images#
for i in range(4):


#
if withBB:
#  FILL ME#  print bounding box shapes and first N entries




## Additional Exercise
3. If we have more categories (let say N) we should use a categorical label that is a vector of length N with 1 on the category(/ies) the image belong to and 0 in the others. Try to build a categorical label for two categories

4. Expand the categorical label to  Ellispes or  Lines  and possibly also non exclusive categories such has 2D vs 1D objects

In [ ]:
#QUESTION# What are the tensors now corresponding to "X" and "y"  ?
if withBB :
    y=[targets,boundingBoxes]
else:
    y=targets

X=images

# Let's build a CNN

Now we build our first CNN. We have some Conv layers interleaved with MaxPooling, and finally we flatten the output of the convolutional stack and appply a Dense FF:
![alt text](https://missinglink.ai/wp-content/uploads/2019/08/LeNet-5-1998.png)


### MaxPooling
![alt text](https://computersciencewiki.org/images/8/8a/MaxpoolSample2.png)

### Additional Exercise
5. Try adding/removing convolutional layers, change the kernel size, try to add dropout
6. Try changing the model to categorical labels, change loss function from binary_crossentropy to categorical_crossentropy, and use softmax activation instead of sigmoid


In [ ]:
noConv = False


inputs=Input(shape=(64,64,3 )) #FILL ME# What is the input shape?
if not noConv :
# try building a CNN with three 2D Conv layers with a number of filters between 3 and 10
# interleave them with with Max  pooling layers
# then use the "Flatten" layer to forget the multidimensional shape
else :
  #if we do not use convolution, we can just flatten the input tensor as first layer
  hidden= Flatten()(inputs)

# after the convolutional part we use standard FeedForward dense layers to further process
hidden=  Dense(50, activation='relu')(hidden)
hidden=  Dense(20, activation='relu')(hidden)
hidden=  Dense(10, activation='relu')(hidden)

#FILL ME# now let's try to decide the output layer, the activation and the loss based on the problem we have
if simple :
  outputs =  #FILLME# what is the output shape in this case? which activation?
  loss=""  # [0,1] classification hence loss should be => .... ?
if mixed :
  outputs = None #FILL ME#  Here we want to predict how many rect? and how many Circ?
  loss='' #FILLME#
if withBB : #In this case we have two separate output layers, two target tensors, two losses
  output1 = Dense(1, activation='sigmoid')(hidden) #FILLME# which activation for the circle vs rectangle classification?
  output2 = Dense(4)(hidden) #FILLME# which activation to predict the 4 numbers of the bounding box ?
  outputs=[output1,output2]
  loss=["",""] #FILL ME # And which loss for each of the two outputs? (the final loss will be the sum unless we give them some additional weights)


model = Model(inputs=inputs, outputs=outputs)
model.compile(loss=loss, optimizer='adam',metrics=['accuracy'])

model.summary()
from tensorflow.keras.utils import plot_model
if withBB:
#FILL ME#  try to print the graph of the network in this case
   plot_model(model, show_shapes=True, show_layer_names=True, to_file='model.png')
   from IPython.display import Image
   Image(retina=True, filename='model.png')

And now let's fit it to our data.
The sample is automatically split in two so that 50% of it is used for validation and the other half for training


In [ ]:
history=model.fit(X,y,validation_split=0.5,epochs=150) #FILL ME# Who are "X" and "y" here?

*history* contains information about the training.  We can now now show the loss vs epoch for both validation and training samples.




In [ ]:
print(history.history.keys())
plt.plot(history.history["val_loss"])
plt.plot(history.history["loss"])
plt.show()
#try plotting the separate loss components in the case of the bounding box problem
#plt.plot(history.history[...]
#plt.plot(history.history[...]
#plt.plot(history.history[...]
#plt.plot(history.history[...]
#plt.yscale('log')

#for classification problem you can plot also the accuracy metric
#plt.plot(history.history["val_accuracy"])
#plt.plot(history.history["accuracy"])
#plt.show()


Let's try to create a new image and evaluate it

In [ ]:
#FILL ME# try to generate a new circle and check if the network recognizes it

print("It is a","circle" if #FILL ME# else "rectangle" )


if mixed :
  im= genCircle()[0]+genRectangle()[0]+genRectangle()[0]
  #FILL ME#try predicting on the "im" generated above... how do we access the number of predicted objects?
  #print("In this image there are %s rectangle and %s circles"%(....,....))
  plt.imshow(im)
  plt.show()

if withBB:
  im,x1,y1,x2,y2= genCircle()
  pre=model.predict(np.expand_dims(im,axis=0) )
  isCircle=pre[0][0]
  x1p=pre[1][0][0]-1.
  y1p=pre[1][0][1]-1.
  x2p=pre[1][0][2]+1.
  y2p=pre[1][0][3]+1.
  print(x1p,y1p,x2p,y2p)
  #print("Looks like a", ("circle" if model.predict(np.expand_dims(im,axis=0) )[0,1] > 0.5 else "rectangle"))
  print("It is a","circle" if isCircle > 0.5 else "rectangle" )
  cv2.rectangle(im,(x1-1,y1-1),(x2+1,y2+1), (255,0,0), 1)
  cv2.rectangle(im,(int(x1p),int(y1p)),(int(x2p),int(y2p)), (255,255,0), 1)
  plt.imshow(im)
  plt.show()
